In [1]:
import os
import timeit
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.sparse import random
import numpy as np
from scipy.sparse import coo_matrix
from scipy.io import mmread
#from PE import PE # cycle counting
from time import perf_counter

In [2]:
def RowPartitioning(M, peSize):
    row, column = M.shape
    count = 0
    matrixList = []
    for i in range(row):
        if(count == 0):
            part = np.vstack(M[i:i+1])
            count += 1
            if(i == row - 1):
                matrixList.append(part)
        elif(count < peSize):
            part = np.vstack((part,M[i:i+1]))
            count += 1
            if(i == row - 1):
                matrixList.append(part)
        elif(count == peSize):
            count = 1
            matrixList.append(part)
            part = np.vstack(M[i:i+1])
            if(i == row - 1):
                matrixList.append(part)
    return matrixList

In [3]:
def columnSplit(M,label,peSize):
    r,c = M.shape
    level = []
    levelLabel = []
    x = int(c/peSize)
    for i in range(x):
        subMatrix = M[:,i*peSize:peSize*(i+1)]
        subLabel = label[:,i*peSize:peSize*(i+1)]
        level.append(subMatrix)
        levelLabel.append(subLabel)
    return level,levelLabel

In [4]:
def delColumns(M, label):
        l = []
        r, c = M.shape
        for i in range(c):
            zero = 1
            for j in range(r):
                if (M[..., i][j] != 0):
                    zero = 0
                    break;
            if (zero == 0):
                l.append(i)
        M_list = []
        flag_M_list = []
        for i in range(len(l)):
            M_list.append(M[..., l[i]])
            flag_M_list.append(label[..., l[i]])
        delM = np.array(M_list).T
        delLabel = np.array(flag_M_list).T
        return delM, delLabel

In [5]:
def Mapping(delPackedCombinedMatrixList, delPackedCombinedLabelList):
    mapeddelPackedCombinedLabelList = []
    mapeddelPackedCombinedMatrixList = []
    for i in range(len(delPackedCombinedMatrixList)):
        mapeddelPackedCombinedLabelList.append([])
        mapeddelPackedCombinedMatrixList.append([])
    for i in range(len(delPackedCombinedMatrixList)):
        r, c = delPackedCombinedMatrixList[i].shape
        res = c % peSize[1]
        if (res != 0):
            res = peSize[1] - res
            l, label = [], []
            for j in range(r):
                l.append(0)
                label.append(-1)
            l = np.array(l)
            l = l.reshape(-1, 1)
            label = np.array(label)
            label = label.reshape(-1, 1)
            for j in range(res):
                delPackedCombinedMatrixList[i] = np.hstack((delPackedCombinedMatrixList[i], l))
                delPackedCombinedLabelList[i] = np.hstack((delPackedCombinedLabelList[i], label))
            mapMatrices, mapLabels = columnSplit(delPackedCombinedMatrixList[i], delPackedCombinedLabelList[i],
                                                 peSize[1])
            for item in mapMatrices:
                mapeddelPackedCombinedMatrixList[i].append(item)
            for item in mapLabels:
                mapeddelPackedCombinedLabelList[i].append(item)
        else:
            mapMatrices, mapLabels = columnSplit(delPackedCombinedMatrixList[i], delPackedCombinedLabelList[i],
                                                 peSize[1])
            for item in mapMatrices:
                mapeddelPackedCombinedMatrixList[i].append(item)
            for item in mapLabels:
                mapeddelPackedCombinedLabelList[i].append(item)
    return mapeddelPackedCombinedMatrixList, mapeddelPackedCombinedLabelList

In [6]:
def MappingForSTPU(partitionPackedList):
    for i in range(len(partitionPackedList)):
        r,c = partitionPackedList[i].M.shape
        x = c % peSize[1]
        if(x != 0):
            l,label = [],[]
            for j in range(r):
                l.append(0)
                label.append(-1)
            l = np.array(l)
            l = l.reshape(-1,1)
            label = np.array(label)
            label = label.reshape(-1,1)
            for j in range(peSize[1]-x):
                partitionPackedList[i].M = np.hstack((partitionPackedList[i].M,l))
                partitionPackedList[i].label = np.hstack((partitionPackedList[i].label,label))

    partitionPackedAndColumnSplitedList = []
    partitionPackedAndColumnSplitedListLabel = []
    for i in range(len(partitionPackedList)):
        level , LevelLabel = columnSplit(partitionPackedList[i].M,partitionPackedList[i].label,peSize[1])
        partitionPackedAndColumnSplitedList.append(level)
        partitionPackedAndColumnSplitedListLabel.append(LevelLabel)
    return partitionPackedAndColumnSplitedList,partitionPackedAndColumnSplitedListLabel

In [7]:
# print("Start time Output: ", StartTime) #start time


stationary = random(101 , 100,density=0.2, random_state=None, data_rvs=None) #change here
stationary = stationary.toarray()
#print(stationary)
streaming = random(100 , 101, density=0.1, random_state=None, data_rvs=None) #change here
streaming = streaming.toarray()

peSize = (8,8) #change here

In [8]:
StartTime_S = perf_counter()
# print("Start Time for S:", StartTime_S)

class Matrix:
    N_ITER, MAX_COLLISION_ROW, MAX_COLLISION_GROUP = 2, 1, [15, 4]

    def __init__(self, M):
        self.M = M
        self.r, self.c = M.shape
        # print(r,c)
        self.label = np.zeros((self.r, self.c))
        self.flag = []
        for i in range(self.c):
            for j in range(self.r):
                if (M[..., i][j] != 0):
                    self.label[..., i][j] = i
                else:
                    self.label[..., i][j] = -1
        for i in range(self.c):
            self.flag.append(1)

    def greedyAlgorithm(self):
        for i in range(self.c):
            self.flag[i] = 1
        for i in range(self.c):
            if (int(self.flag[i]) == 1):
                for j in range(i + 1, self.c):
                    collision = 0
                    for k in range(self.r):
                        if (self.M[..., i][k] != 0 and self.M[..., j][k] != 0):
                            collision = 1
                            break;
                    if (collision == 0):
                        self.M[..., i] = self.M[..., i] + self.M[..., j]
                        self.M[..., j] = self.M[..., j] - self.M[..., j]
                        for k in range(self.r):
                            if (self.label[..., i][k] == -1 and self.label[..., j][k] != -1):
                                self.label[..., i][k], self.label[..., j][k] = self.label[..., j][k], self.label[..., i][k]
                        self.flag[j] = 0
            else:
                continue

    def __collisionRelax(self):
        for iteration in range(self.N_ITER):
            self.__collitoinPacking(iteration)

    def __collitoinPacking(self, iteration):
        for i in range(self.c):
            self.flag[i] = 1
        if (iteration == 0):
            maxCollisionGroup = self.MAX_COLLISION_GROUP[0]
        else:
            maxCollisionGroup = self.MAX_COLLISION_GROUP[1]
        for i in range(self.c):
            countForCollisions = 0
            if (int(self.flag[i]) == 1):
                for j in range(i + 1, self.c):
                    if (self.flag[j] == 1):
                        collision = 0
                        for k in range(self.r):
                            if (self.M[..., i][k] != 0 and self.M[..., j][k] != 0):
                                collision += 1
                            if (collision == self.MAX_COLLISION_ROW + 1):
                                break
                        if (collision > self.MAX_COLLISION_ROW):
                            continue
                        else:
                            for k in range(self.r):
                                if (self.M[..., i][k] == 0 and self.M[..., j][k] != 0):
                                    self.M[..., i][k] = self.M[..., j][k]
                                    self.M[..., j][k] = 0
                                    self.label[..., i][k] = self.label[..., j][k]
                                    self.label[..., j][k] = -1
                            if (collision != 0):
                                countForCollisions += 1
                            self.flag[j] = 0
                        if (countForCollisions == maxCollisionGroup):
                            break

    def collisionAwareAlgorithm(self):
        self.__collisionRelax()
        self.greedyAlgorithm()

    def delColumns(self):
        l = []
        for i in range(self.c):
            zero = 1
            for j in range(self.r):
                if (self.M[..., i][j] != 0):
                    zero = 0
                    break;
            if (zero == 0):
                l.append(i)
        M_list = []
        flag_M_list = []
        for i in range(len(l)):
            M_list.append(self.M[..., l[i]])
            flag_M_list.append(self.label[..., l[i]])
        self.M = np.array(M_list).T
        self.label = np.array(flag_M_list).T
        
def Spack(stationary,streaming):
    r,c = stationary.shape
    label=np.zeros((r,c))
    for i in range(c):
        for j in range(r):
            if(stationary[...,i][j]!=0):
                label[...,i][j]=i
            else:
                label[...,i][j]=-1
    partitionList = RowPartitioning(stationary,peSize[0])
    partitionLabelList = RowPartitioning(label,peSize[0])
    partitionPackedList = []
    count = 0
    for item in partitionList:
        itemM = Matrix(item)
        itemM.label = partitionLabelList[count]
        count += 1
        itemM.collisionAwareAlgorithm()
        itemM.delColumns()
        partitionPackedList.append(itemM)
    #streaming= np.transpose(streaming)
    r,c = streaming.shape
    label = np.zeros((r,c))
    for i in range(c):
        for j in range(r):
            if(streaming[...,i][j]!=0):
                label[...,i][j]=j
            else:
                label[...,i][j]=-1
    #print(label)
    streamingPartitionList = RowPartitioning(streaming.T,peSize[0])
    streamingPartitionLabelList = RowPartitioning(label.T,peSize[0])
    count= 0
    streamingPartitionPackedList = []
    for item in streamingPartitionList:
        itemM = Matrix(item)
        itemM.label =streamingPartitionLabelList[count]
        count += 1
        itemM.collisionAwareAlgorithm()
        itemM.delColumns()
        streamingPartitionPackedList.append(itemM)
    for i in range(len(streamingPartitionPackedList)):
        streamingPartitionPackedList[i].M = streamingPartitionPackedList[i].M.T
        streamingPartitionPackedList[i].label = streamingPartitionPackedList[i].label.T
    partitionPackedAndColumnSplitedList,partitionPackedAndColumnSplitedListLabel = MappingForSTPU(partitionPackedList)
    return partitionPackedAndColumnSplitedListLabel,streamingPartitionPackedList
partitionPackedAndColumnSplitedListLabel,streamingPartitionPackedList = Spack(stationary,streaming)



ENDTIME_S = perf_counter()
# print("End time for S:", ENDTIME_S)
print("S_Output Total Time:", ENDTIME_S-StartTime_S)


S_Output Total Time: 0.3085613659968658


In [9]:
partitionPackedAndColumnSplitedListLabel

[[array([[ 3., 25., 19., 47.,  5., 48., 54., 34.],
         [ 7., 11., 24., 26., 25., -1., -1., 54.],
         [ 7., 10., 18., 31., 26., -1., -1., 20.],
         [14., 29., 15., 36., 12., 49., 61., 35.],
         [ 0., 23., 22., 35., 12., 42., 43., 16.],
         [ 2.,  7., 12., 32.,  8., 34., 37., 20.],
         [ 0.,  1.,  2.,  3.,  5.,  6.,  9., 34.],
         [ 0., 20., 13., 38.,  8., 57., 70., 16.]]),
  array([[36., 59., 68., 45., 71., 73., 90., 55.],
         [31., 44., -1., 71., -1., 57., -1., 98.],
         [28., -1., -1., -1., -1., 56., -1., -1.],
         [28., 39., 63., 48., 69., 52., 72., 55.],
         [31., 39., 59., 53., 87., 57., -1., 65.],
         [28., 44., 41., 48., 50., 52., 53., 55.],
         [49., 68., 40., 70., 95., 75., 98., 77.],
         [49., 44., 79., 48., 80., 51., 88., 65.]]),
  array([[61., 81., -1., -1., -1., -1., 83., 87.],
         [61., 81., -1., -1., -1., -1., 78., 89.],
         [69., -1., -1., -1., -1., -1., 78., -1.],
         [60., 62., 75., 82

In [10]:
streaminglabelstpu=[]
for i in range(len(streamingPartitionPackedList)):
  streaminglabelstpu.append(streamingPartitionPackedList[i].label)
print(streaminglabelstpu)

[array([[14.,  4.,  8.,  6., 37., 20.,  4.,  7.],
       [20., 10., 15., 16., 53., 27., 28., 40.],
       [23., 33., 25., 27., 23., 23., 60., 25.],
       [90., 26., -1., 61., 56., -1., 29., 95.],
       [39., 30., 48., 43., 47., 41., 37., 33.],
       [54., 52., -1., 44., 44., 52., 71., 54.],
       [98., 57., -1., -1., 79., -1., 64., -1.],
       [-1., 78., -1., -1., 83., -1., 65., -1.],
       [-1., 83., 74., 66., 66., -1., 66., 71.],
       [-1., -1., -1., -1., -1., -1., 68., -1.],
       [-1., -1., -1., -1., -1., -1., 77., -1.],
       [84., 84., -1., 84., 81., -1., 81., 93.],
       [-1., -1., -1., -1., -1., -1., 86., -1.],
       [-1., -1., -1., -1., -1., -1., 88., -1.]]), array([[26.,  1., 11.,  0.,  5.,  2.,  8.,  8.],
       [51., 92., 17., 12., 11., 19., 16., 13.],
       [55., -1., 18., 45., 14., -1., 23., 61.],
       [22., 45., 16., 15., 37., 22., 15., 37.],
       [32., 34., 30., 22., -1., 24., 21., 27.],
       [56., -1., 33., 47., -1., -1., 49., 83.],
       [43., 76.,

In [11]:
for item in streaminglabelstpu:
    print(item.shape)

(14, 8)
(12, 8)
(17, 8)
(14, 8)
(14, 8)
(18, 8)
(17, 8)
(16, 8)
(14, 8)
(15, 8)
(16, 8)
(12, 8)
(11, 5)


In [12]:
def addBlankLine(m):
    while(m.shape[0] < peSize[0]):
        blank = np.array([-1 for i in range(m.shape[1])])
        m = np.vstack([m , blank])
    return m
# m = np.random.rand(2, 8)
# m= addBlankLine(m)
# print(m)


In [13]:
streaminglabelMappedstpu = []
for item in streaminglabelstpu:
    streamingRowSplit = RowPartitioning(item, peSize[0])
    for i in range(len(streamingRowSplit)):
        streamingRowSplit[i] = addBlankLine(streamingRowSplit[i])
    streaminglabelMappedstpu.append(streamingRowSplit)


In [14]:
cnt = 1
for streaming_l in streaminglabelMappedstpu:
    print("level " + str(cnt))
    for item in streaming_l:
        print(item.shape)
    

level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
level 1
(8, 5)
(8, 5)


In [15]:
# streaminglabelMappedstpu

In [16]:
print(len(streaminglabelstpu))
for item in streaminglabelstpu:
    print(item.shape)

13
(14, 8)
(12, 8)
(17, 8)
(14, 8)
(14, 8)
(18, 8)
(17, 8)
(16, 8)
(14, 8)
(15, 8)
(16, 8)
(12, 8)
(11, 5)


In [17]:
print(len(partitionPackedAndColumnSplitedListLabel))
print(len(partitionPackedAndColumnSplitedListLabel[0]))

13
4


In [18]:
for i in range(len(partitionPackedAndColumnSplitedListLabel)):
    print("level " + str(i))
    for j in range(len(partitionPackedAndColumnSplitedListLabel[i])):
        print(partitionPackedAndColumnSplitedListLabel[i][j].shape)

level 0
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 1
(8, 8)
(8, 8)
(8, 8)
level 2
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 3
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 4
(8, 8)
(8, 8)
(8, 8)
level 5
(8, 8)
(8, 8)
(8, 8)
level 6
(8, 8)
(8, 8)
(8, 8)
level 7
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 8
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 9
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 10
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 11
(8, 8)
(8, 8)
(8, 8)
(8, 8)
level 12
(5, 8)
(5, 8)
(5, 8)


In [21]:
for i in range(len(streaminglabelstpu)):
    np.savetxt("./data/stpu/streaming/steaming" + str(i), streaminglabelstpu[i], fmt="%d")

In [27]:
def mkdir(path):
	folder = os.path.exists(path)
	if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
		os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
# file = "G:\\xxoo\\test"
# mkdir(file)        
path = "./data/stpu/stationary/"
level = len(partitionPackedAndColumnSplitedListLabel)
for i in range(level):
	mkdir(path + "level"+str(i))
	for j in range(len(partitionPackedAndColumnSplitedListLabel[i])):
		print(j)
		np.savetxt(path + "level" + str(i) + "/" + "stationary" + str(j), partitionPackedAndColumnSplitedListLabel[i][j], fmt="%d")




0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
1
2
0
1
2
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3
0
1
2
3
0
1
2
3
0
1
2
